In [10]:
from functions import retrieve_data
from pandas import read_parquet, DataFrame, concat, read_pickle
from tqdm import tqdm
from time import sleep
from glob import glob

from os import getenv

from dotenv import load_dotenv
load_dotenv()

True

Rate limit with key:

    1 request per second for the following endpoints:
        /paper/batch
        /paper/search
        /recommendations
    10 requests / second for all other calls

 Additionally, all unauthenticated users share a limit of 5,000 requests per 5 minutes. 

In [2]:
## Sugiero que cada quien cree su propia API key, y de esta manera aumentar 
## el volumen de requests por segundo
API_KEY = getenv('API_KEY')

## Retrieve Data of DB sample

In [3]:
#df = read_json('data/dict_split_4.json', orient='index')
#df.reset_index(inplace=True, names=['db_id'])
#df['db_id'] = df['db_id'].astype(str)
#df.to_parquet('data/data_4.parquet', index=False)

In [11]:
df = read_parquet('data/data_4.parquet', 
                  columns=['db_id','title','year']
                  )  
len(df)

156240

In [12]:
files = glob('data/api_request_results/documents/*')
files = [f.split('/')[-1][5:-4] for f in files]

In [13]:
df = df[~df['db_id'].isin(files)]
df = df.sample(len(df), ignore_index=True)
len(df)

153244

In [7]:
for row in tqdm(df.itertuples()):
    
    res = retrieve_data.request_semantic_scholar(row.title,
                                                 api_key=API_KEY, 
                                                 fields=retrieve_data.QUERY_FIELDS)
    
    if res['status_code']==200:
        data = res.get('data')
        if data:
            df_res = DataFrame.from_dict(data)
            
            df_res['db_id'] = row.db_id
            df_res['db_title'] = row.title
            df_res['db_year'] = row.year
            
            df_res.to_pickle(f'data/api_request_results/documents/data_{row.db_id}.zip', 
                             compression= {
                                'method': 'zip',
                                'compresslevel': 9  # Nivel máximo de compresión para ZIP
                                }
                            )
        else:
            print(f'No data for {row.title} ({row.db_id})')
        sleep(1)
    elif res['status_code']==429:
        print('Too many requests in {row.title} ({row.db_id}). Waiting 180 seconds')
        sleep(180)
    else: 
        print('Error:', res)
        break
        

6it [01:33, 16.08s/it]

No data for Human-Robot Site Survey and Sampling for Space Exploration, AIAA-2006-7425 (3913693)


26it [07:11, 29.25s/it]

In [14]:
retrieved_data = concat([read_pickle(f) for f in tqdm(glob('data/api_request_results/documents/*'))])

100%|██████████| 2996/2996 [00:12<00:00, 236.14it/s]
/tmp/ipykernel_5366/1332076495.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  retrieved_data = concat([read_pickle(f) for f in tqdm(glob('api_request_results/documents/*'))])


In [15]:
retrieved_data.to_pickle(f'data/api_request_results/retrieved_data.zip', 
                             compression= {
                                'method': 'zip',
                                'compresslevel': 9  # Nivel máximo de compresión para ZIP
                                }
                        )

In [ ]:
'''
url = 'https://bitcoin.org/bitcoin.pdf'
response = get_request(url)
with open('sample.pdf', 'wb') as f:
    f.write(response.content)
'''   